In [1]:
!pip install torch
!pip install transformers
!pip install sentencepiece

In [38]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# sklearn.model_selection 모듈의 train_test_split 함수는 데이터를 학습 데이터와 테스트 데이터로 분할하는데 사용됩니다.
from sklearn.model_selection import train_test_split

# transformers의 BertTokenizer, AdamW, BertModel은 BERT 모델(자연어처리를 위한 트랜스포머 기반 모델)을 사용하는데 필요한 클래스입니다.
from transformers import BertTokenizer, AdamW, BertModel

# torch.utils.data의 Dataset, DataLoader는 파이토치(PyTorch)에서 데이터를 로드하고 전처리하는데 사용되는 클래스입니다.
from torch.utils.data import Dataset, DataLoader

# nn과 torch는 파이토치 라이브러리로, 딥러닝 모델을 구현하는데 사용됩니다. nn은 신경망을 구성하는데 필요한 다양한 모듈과 손실 함수를 제공하며, torch는 텐서 등의 기본 연산을 제공합니다.
import torch.nn as nn
import torch

# sklearn.metrics의 accuracy_score, f1_score, precision_score, recall_score는 모델의 성능을 평가하는데 사용되는 메트릭 함수입니다.
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [39]:
# csv 파일을 읽어오기
data = pd.read_csv("restaurant_data.csv")

In [40]:
data_only = data[["발화문", "인텐트"]]

In [41]:
# transformers 라이브러리에서 BertTokenizerFast를 임포트합니다.
# BertTokenizerFast는 BERT 토크나이저의 빠른 버전입니다.
from transformers import BertTokenizerFast

# 'bert-base-uncased'라는 이름의 사전 훈련된 모델을 사용하여 토크나이저를 초기화합니다.
# 'bert-base-uncased'는 가장 기본적인 BERT 모델입니다.
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# data_only['발화문'].tolist()를 사용하여 토크나이저를 통해 데이터를 인코딩합니다.
# truncation=True는 문장이 모델의 최대 문장 길이를 초과할 경우 잘라내는 역할을 합니다.
# padding=True는 모든 문장을 모델의 최대 문장 길이에 맞게 패딩하는 역할을 합니다.
encodings = tokenizer(data_only['발화문'].tolist(), truncation=True, padding=True)


In [42]:
# sklearn.preprocessing 모듈의 LabelEncoder를 임포트합니다.
# LabelEncoder는 범주형 변수를 정수로 인코딩하는데 사용되는 클래스입니다.
from sklearn.preprocessing import LabelEncoder

# LabelEncoder 인스턴스를 생성합니다.
le = LabelEncoder()

# '인텐트'라는 열에 대해 fit_transform 메소드를 적용하여 범주형 라벨을 정수로 인코딩합니다.
# fit_transform 메소드는 fit 메소드(라벨에 대한 인코딩 규칙을 학습)와 transform 메소드(해당 규칙을 데이터에 적용)를 순차적으로 수행합니다.
data_only['인텐트'] = le.fit_transform(data_only['인텐트'])


/tmp/ipykernel_30607/260234296.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_only['인텐트'] = le.fit_transform(data_only['인텐트'])


In [43]:
# 인코딩된 데이터에서 'input_ids'를 가져와 input_ids 변수에 저장합니다.
# 'input_ids'는 토큰화된 각 토큰을 인덱스로 변환한 결과를 담고 있습니다.
input_ids = encodings['input_ids']

# 인코딩된 데이터에서 'attention_mask'를 가져와 attention_masks 변수에 저장합니다.
# 'attention_mask'는 패딩된 부분과 실제 토큰화된 부분을 구분하기 위한 마스크입니다.
attention_masks = encodings['attention_mask']

# '인텐트' 열의 값을 numpy 배열로 변환하여 labels 변수에 저장합니다.
# 이는 모델 학습에 사용될 타겟 값(라벨)입니다.
labels = data_only['인텐트'].to_numpy()


In [44]:
# PyTorch의 Dataset 클래스를 상속받아 MenuDataset 클래스를 정의합니다.
# 이 클래스는 BERT 모델에 입력될 데이터를 관리하고, 각각의 데이터를 반환하는 기능을 제공합니다.
class MenuDataset(Dataset):
    # 초기화 메소드에서는 input_ids, attention_masks, labels를 인자로 받아 클래스의 멤버 변수로 저장합니다.
    # input_ids와 attention_masks는 리스트 형태의 데이터를 PyTorch의 tensor 데이터로 변환하여 저장합니다.
    def __init__(self, input_ids, attention_masks, labels):
        self.input_ids = [torch.tensor(ids, dtype=torch.long) for ids in input_ids]
        self.attention_masks = [torch.tensor(mask, dtype=torch.long) for mask in attention_masks]
        self.labels = labels

    # __len__ 메소드는 데이터셋의 총 데이터 개수를 반환합니다.
    def __len__(self):
        return len(self.input_ids)

    # __getitem__ 메소드는 주어진 인덱스에 해당하는 데이터를 반환합니다.
    # 여기서는 input_ids, attention_mask, labels를 포함한 딕셔너리 형태로 반환하고 있습니다.
    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_masks[idx],
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }


In [45]:
# 데이터셋 생성
dataset = MenuDataset(input_ids=input_ids, attention_masks=attention_masks, labels=labels)

# 데이터셋을 학습용과 검증용으로 분리
train_data, val_data = train_test_split(dataset, test_size=0.2)

In [46]:
# hyper parmeter

# 전체 데이터셋에 대한 학습 반복 회수
epochs= 5
# 한 번의 학습 단계에서 사용할 데이터의 개수
batch_size=32 
# learning_rate(학습률)
# 경사 하강법에서 한 스텝의 크기를 결정
lr = 5e-5 

In [47]:
# DataLoader 생성
train_data_loader = DataLoader(train_data, batch_size=batch_size)
val_data_loader = DataLoader(val_data, batch_size=batch_size)

In [64]:
# PyTorch의 nn.Module 클래스를 상속받아 BertClassifier 클래스를 정의합니다.
# 이 클래스는 BERT 모델을 이용한 분류 모델을 구현한 것입니다.
class BertClassifier(nn.Module):
    # 초기화 메소드에서는 분류할 라벨의 개수(num_labels)와 드롭아웃 비율(dropout_rate)를 인자로 받습니다.
    # 'bert-base-uncased'라는 이름의 사전 훈련된 모델을 사용하여 BERT 모델을 초기화하고,
    # 드롭아웃 레이어와 선형 레이어를 추가합니다.
    def __init__(self, num_labels, dropout_rate=0.3):
        super(BertClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = nn.Dropout(dropout_rate)
        self.linear = nn.Linear(768, num_labels)

    # forward 메소드는 입력 데이터(input_ids, attention_mask)를 받아 BERT 모델을 통과시키고,
    # 그 결과를 드롭아웃 레이어와 선형 레이어를 통과시켜 최종 결과를 반환합니다.
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs['pooler_output']
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        return linear_output


In [49]:
# '인텐트' 열의 라벨 개수를 구하고, 이를 사용하여 BertClassifier 모델을 생성합니다.
num_labels = len(np.unique(data_only['인텐트']))
model = BertClassifier(num_labels)

# GPU를 사용하기 위해 디바이스를 설정하고, 모델을 해당 디바이스로 이동시킵니다.
device = torch.device('cuda')
model = model.to(device)

# 옵티마이저와 손실 함수를 설정합니다. 
# 여기서는 AdamW 옵티마이저와 크로스 엔트로피 손실 함수를 사용합니다.
optimizer = AdamW(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss().to(device)

# 주어진 에폭 수만큼 학습을 반복합니다.
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    
    # 학습 모드로 설정합니다.
    model.train()
    
    # 학습 데이터 로더에서 배치 단위로 데이터를 가져옵니다.
    for batch in tqdm(train_data_loader, desc="Training"):
        # 배치에서 데이터를 가져와 GPU로 이동시킵니다.
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # 모델에 입력 데이터를 넣어 출력 결과를 얻습니다.
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        
        # 손실 함수에 출력 결과와 타깃 라벨을 넣어 손실 값을 계산합니다.
        loss = criterion(outputs, labels)

        # 경사를 초기화하고, 역전파를 수행하고, 가중치를 업데이트합니다.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # 검증 모드로 설정합니다.
    model.eval()
    
    # 검증 데이터에 대한 손실 값을 저장할 리스트와 예측 결과 및 실제 라벨을 저장할 리스트를 생성합니다.
    val_losses = []
    val_predictions = []
    val_truths = []
    
    # 검증 데이터 로더에서 배치 단위로 데이터를 가져옵니다.
    for batch in tqdm(val_data_loader, desc="Validating"):
        # 배치에서 데이터를 가져와 GPU로 이동시킵니다.
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
    
        # 그래디언트 계산을 비활성화하고, 모델에 입력 데이터를 넣어 출력 결과를 얻습니다.
        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            val_loss = criterion(outputs, labels)
    
        # 손실 값을, 예측 결과, 실제 라벨을 저장합니다.
        val_losses.append(val_loss.item())
        val_predictions.extend(torch.argmax(outputs, dim=1).cpu().detach().numpy().tolist())
        val_truths.extend(labels.cpu().detach().numpy().tolist())
    
    # 평균 손실 값을, 정확도, F1 점수, 정밀도, 재현율을 계산합니다.
    val_loss = sum(val_losses) / len(val_losses)
    val_acc = accuracy_score(val_truths, val_predictions)
    val_f1 = f1_score(val_truths, val_predictions, average='weighted')
    val_precision = precision_score(val_truths, val_predictions, average='weighted')
    val_recall = recall_score(val_truths, val_predictions, average='weighted')
    
    # 계산된 메트릭 값을 출력합니다.
    print(f"Validation Loss: {val_loss:.4f} Accuracy: {val_acc:.4f} F1-score: {val_f1:.4f} Precision: {val_precision:.4f} Recall: {val_recall:.4f}")


/home/ubuntu/.local/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/100


Validating: 100%|██████████| 1815/1815 [02:22<00:00, 12.73it/s]
/home/ubuntu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation Loss: 2.0636 Accuracy: 0.4731 F1-score: 0.4493 Precision: 0.4631 Recall: 0.4731
Epoch 2/100


Validating: 100%|██████████| 1815/1815 [02:22<00:00, 12.73it/s]
/home/ubuntu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

Validation Loss: 1.8018 Accuracy: 0.5301 F1-score: 0.5193 Precision: 0.5433 Recall: 0.5301
Epoch 6/100


Validating: 100%|██████████| 1815/1815 [02:22<00:00, 12.73it/s]
/home/ubuntu/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation Loss: 1.8011 Accuracy: 0.5313 F1-score: 0.5238 Precision: 0.5478 Recall: 0.5313
Epoch 7/100


Training:  81%|████████  | 5872/7260 [23:00<05:26,  4.25it/s]


KeyboardInterrupt: 

In [65]:
def predict_intent(text, model, tokenizer):
    # 텍스트를 토크나이즈하고 BERT 입력 형식에 맞게 변환
    inputs = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        return_tensors='pt',
        return_token_type_ids=False
    )
    
    # 각 텐서를 GPU로 이동
    inputs = {name: tensor.to(device) for name, tensor in inputs.items()}
    
    # 모델의 예측 생성
    with torch.no_grad():
        outputs = model(**inputs)
    
    # 가장 높은 확률을 가진 클래스의 인덱스를 가져옴
    _, predicted = torch.max(outputs, dim=1)
    
    # 예측된 인덱스를 의도로 변환
    # 이 부분은 실제 의도와 인덱스를 매핑하는 방법에 따라 다르게 작성해야 합니다.
    intent = le.inverse_transform([predicted.item()])[0]
    
    return intent


In [73]:
# 테스트
text = input()
predicted_intent = predict_intent(text, model, tokenizer)
print(predicted_intent)

 배달료 따로 있나요?


배송_비용_질문


In [75]:
# 모델의 state_dict 저장
torch.save(model.state_dict(), "/home/ubuntu/Project/HG/saved_model1/Bert_model1.pth")

In [76]:
# LabelEncoder 저장
with open("/home/ubuntu/Project/HG/saved_model1/label_encoder.pkl", "wb") as f:
    pickle.dump(le, f)

NameError: name 'pickle' is not defined